In [1]:
from simply_nwb.pipeline import NWBSession
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
sess = NWBSession("predicted.nwb")

Reading NWB file 'predicted.nwb'..


In [3]:
sess.available_enrichments()

['PredictSaccades', 'PutativeSaccades']

In [5]:
sess.available_keys("PredictSaccades")

['saccades_fps',
 'saccades_predicted_nasal_epochs',
 'saccades_predicted_nasal_peak_indices',
 'saccades_predicted_nasal_waveforms',
 'saccades_predicted_noise_waveforms',
 'saccades_predicted_temporal_epochs',
 'saccades_predicted_temporal_peak_indices',
 'saccades_predicted_temporal_waveforms']

In [8]:
sess.description("PutativeSaccades")

{'raw_x': 'x values directly from the DLC csv (time,)',
 'raw_y': 'y values directly from the DLC csv (time,)',
 'raw_timestamps': 'timestamps directly from the timestamps.txt (time,)',
 'raw_likelihoods': 'likelihoods directly from the DLC csv (time,)',
 'pose_corrected': 'Corrected eye position (time, x/y)',
 'pose_interpolated': 'Interpolated eye position (time, x/y)',
 'pose_decomposed': 'PCA Imputed eye positions (time, x/y)',
 'pose_missing': 'Indexes of missing (nan) position values (time, x/y)',
 'pose_reoriented': 'Reoriented positions, so they all start from the same 0 (time, x/y)',
 'processed_eyepos': 'Final interpolation and nan filtering of eye position pose_reoriented (time, x/y)',
 'saccades_putative_peak_indices': 'Index into time of the center (peak velocity) of the saccade (saccadenum,)',
 'saccades_putative_waveforms': 'Waveforms of putative saccades (saccadenum, time, x/y)',
 'saccades_fps': 'single value list like [fps] with the fps used to calculate eye positions

In [13]:
print(sess.get_funclist("PutativeSaccades")[0])

Function 'dropped_frames'
	Args:
		{}
	Desc:
		Number of dropped frames (int)
	Example:
		dropped_frames()


In [16]:
sess.func("PutativeSaccades.dropped_frames")("rightCamTimestamps")

0

In [3]:

eyepos = sess.pull("PutativeSaccades.pose_reoriented")[:, 0]  # corrected, interpolated, reoriented and filtered
# eyepos = sess.pull("PutativeSaccades.raw_x")  # Raw unprocessed DLC (might not be the same length, confirm with above value shape)
nasal_peaks = sess.pull("PredictSaccades.saccades_predicted_nasal_peak_indices")
temporal_peaks = sess.pull("PredictSaccades.saccades_predicted_temporal_peak_indices")

print(f"Eyepos {eyepos.shape}")
print(f"Nasal {nasal_peaks.shape}")
print(f"Temporal {temporal_peaks.shape}")

Eyepos (392576,)
Nasal (128,)
Temporal (302,)


In [4]:
# Plot the eye position with the
fig, axs = plt.subplots(nrows=2, sharex=True)
axs[0].plot(eyepos)
for nas in nasal_peaks:
    axs[1].vlines(nas, -20, 15, color="blue")
for tmp in temporal_peaks:
    axs[1].vlines(tmp, -20, 15, color="orange")
plt.show()


In [5]:
# List available enrichments in this dataset
sess.available_enrichments()

['PutativeSaccades', 'PredictSaccades']

In [6]:
# List out the datasets we have available for a given enrichment
sess.available_keys("PredictSaccades")

['saccades_fps',
 'saccades_predicted_nasal_epochs',
 'saccades_predicted_nasal_peak_indices',
 'saccades_predicted_nasal_waveforms',
 'saccades_predicted_noise_waveforms',
 'saccades_predicted_temporal_epochs',
 'saccades_predicted_temporal_peak_indices',
 'saccades_predicted_temporal_waveforms']

In [8]:
# Grab the saccade waveforms
nasal_waveforms = sess.pull("PredictSaccades.saccades_predicted_nasal_waveforms")

In [9]:
# Shape is (saccade num, time, x&y)
nasal_waveforms.shape

(128, 80, 2)

In [15]:
# So to get all x saccades we can do
nasal_xsaccades = nasal_waveforms[:, :, 0]

In [16]:
# Lets graph the nasal saccades
fig, ax = plt.subplots()
for saccade in nasal_xsaccades:
    ax.plot(saccade)


In [ ]:
del sess

In [13]:
# TEST CODE BUT STILL A GOOD EXAMPLE
sess = NWBSession("drifting.nwb")  # NWB that has been enriched with DriftingGratingLabjackEnrichment(), ie 'DriftingGratingLabjack'
print("nasal_grating_idxs")
print(sess.description('DriftingGratingLabjack')['nasal_grating_idxs'])

print("\nsaccades_predicted_temporal_peak_indices")
print(sess.description('PredictSaccades')['saccades_predicted_temporal_peak_indices'])


Reading NWB file 'drifting.nwb'..
nasal_grating_idxs
Index into the grating windows for each nasal saccade

saccades_predicted_temporal_peak_indices
Indexes of peaks for temporal labeled waveforms (saccadenum,) absolute frames


In [23]:
sess.description("DriftingGratingLabjack")

{'file_len': 'Length of each drifting grating file entries, used to index into the other keys to determine which file contains which range of indexes',
 'filename': 'filenames of each drifingGrating.txt',
 'nasal_grating_idxs': 'Index into the grating windows for each nasal saccade',
 'temporal_grating_idxs': 'Index into the grating windows for each temporal saccade',
 'Spatial frequency': 'How many cycles/degree',
 'Velocity': 'How many degrees/second',
 'Orientation': 'Orientation degrees',
 'Baseline contrast': '0 or 1 for contrast value',
 'Columns': 'Column names for the data',
 'Event (1=Grating, 2=Motion, 3=Probe, 4=ITI)': 'Event data number',
 'Motion direction': 'Motion direction',
 'Probe contrast': 'Contrast number',
 'Probe phase': 'Phase number',
 'Timestamp': 'Timestamp value',
 'Time': 'Labjack times array',
 'v0': 'Labjack channel v0 (currently not used)',
 'v1': 'Labjack channel v1 (currently not used)',
 'v2': 'Labjack channel v2 (currently not used)',
 'v3': 'Labjack

In [26]:
print(sess.pull("DriftingGratingLabjack.Timestamp").shape)
print(sess.pull("DriftingGratingLabjack.Event (1=Grating, 2=Motion, 3=Probe, 4=ITI)").shape)

(4399,)
(4399,)


array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
from simply_nwb.pipeline.util.waves import startstop_of_squarewave


drifting_grating_channel = "y1"
labjack_grating_signal = sess.pull(f"DriftingGratingLabjack.{drifting_grating_channel}")
grating_startstop = startstop_of_squarewave(self.dats[self.frames_channel], **self.squarewave_args)[:, :2] # Chop off the state value, only want start/stop

# video_frame_channel = "y2"  same with this channel instead
